# BNL Development Notebook

This notebook is for developing and testing the BNL package.


In [ ]:
%load_ext autoreload
%autoreload 2

import bnl

# R2_BUCKET_PUBLIC_URL = "https://pub-05e404c031184ec4bbf69b0c2321b98e.r2.dev"

# slm_ds_cloud = bnl.data.Dataset(manifest_path=f"{R2_BUCKET_PUBLIC_URL}/manifest_cloud_boolean.csv")
slm_ds = bnl.data.Dataset(manifest_path="~/data/salami/metadata.csv")
track = slm_ds[8]
est = track.load_annotation("adobe-mu1gamma1")
ref = track.load_annotation("reference")

In [ ]:

# Try loading an Adobe annotation that exists
est = track.load_annotation("adobe-mu1gamma1")
est.plot_plotly(figsize=(600, 400))

In [ ]:
ref = track.load_annotation("reference")
ref.plot_plotly()

In [ ]:
ref.plot_plotly(figsize=None)

In [ ]:
for strategy in ["depth", "count", "prob"]:
    # Using the new fluent API
    contour = est.to_contour(strategy=strategy)
    contour.plot(figsize=(6,3))

    # Convert to hierarchy and then to multisegment for visualization
    hierarchy = contour.to_hierarchy()
    mono_ms = hierarchy.to_multisegment()
    mono_ms.plot(figsize=(6, 3))

In [ ]:
# Demonstrating the new boundary cleaning functionality
print("Original boundary cleaning strategies:")

# Get a boundary contour
contour = est.to_contour(strategy="depth")
print(f"Original contour has {len(contour.boundaries)} boundaries")

# Try different cleaning strategies
for strategy in ["absorb", "kde"]:
    print(f"\nCleaning with {strategy} strategy:")

    if strategy == "absorb":
        # For absorb, we can specify a window parameter
        cleaned = contour.clean(strategy=strategy, window=1.0)
    else:  # kde
        # For kde, we can specify bandwidth and frame_size
        cleaned = contour.clean(strategy=strategy, bandwidth=1.0, frame_size=0.1)

    print(f"  Cleaned contour has {len(cleaned.boundaries)} boundaries")
    cleaned.plot(figsize=(6, 2))


In [ ]:
# Demonstrating the new boundary cleaning functionality
print("Original boundary cleaning strategies:")

# Get a boundary contour
contour = est.to_contour(strategy="depth")
print(f"Original contour has {len(contour.boundaries)} boundaries")

# Try different cleaning strategies
for strategy in ["absorb", "kde"]:
    print(f"\nCleaning with {strategy} strategy:")
    
    if strategy == "absorb":
        # For absorb, we can specify a window parameter
        cleaned = contour.clean(strategy=strategy, window=1.0)
    else:  # kde
        # For kde, we can specify bandwidth and frame_size
        cleaned = contour.clean(strategy=strategy, bandwidth=1.0, frame_size=0.1)
    
    print(f"  Cleaned contour has {len(cleaned.boundaries)} boundaries")
    cleaned.plot(figsize=(6, 2))


In [ ]:
# Demonstrating the new boundary cleaning functionality
print("Original boundary cleaning strategies:")

# Get a boundary contour
contour = est.to_contour(strategy="depth")
print(f"Original contour has {len(contour.boundaries)} boundaries")

# Try different cleaning strategies
for strategy in ["absorb", "kde"]:
    print(f"\nCleaning with {strategy} strategy:")
    
    if strategy == "absorb":
        # For absorb, we can specify a window parameter
        cleaned = contour.clean(strategy=strategy, window=1.0)
    else:  # kde
        # For kde, we can specify bandwidth and frame_size
        cleaned = contour.clean(strategy=strategy, bandwidth=1.0, frame_size=0.1)
    
    print(f"  Cleaned contour has {len(cleaned.boundaries)} boundaries")
    cleaned.plot(figsize=(6, 2))


In [ ]:
# Demonstrating the beautiful fluent API chaining
print("Fluent API chaining - the full pipeline in one expression:")

# The complete pipeline: estimate -> contour -> clean -> hierarchy -> multisegment
final_result = (
    est.to_contour(strategy="depth")
       .clean(strategy="absorb", window=0.5)
       .to_hierarchy()
       .to_multisegment()
)

print(f"Final result has {len(final_result)} layers")
final_result.plot(figsize=(8, 4))
